In [1]:
import pandas as pd
import numpy as np
import requests
import grequests
import os.path

In [2]:
# auxiliary function for below
def split(theList, n):
    for i in range(0, len(theList), n):
        yield theList[i:i + n]

In [3]:
host = "https://api.stratz.com/api/v1/match"

In [4]:
params = {"matchId":[3304258209,3304204784], 
          "include": ["Player"],
         "gameMode": [2, 22],
         "lobbyType": [2, 7]}

In [5]:
x = requests.get(host, params = params)

In [6]:
x.json()

{'results': [{'avgImp': 81,
   'bracket': 4,
   'challenge': 79,
   'clusterId': 122,
   'didRadiantWin': True,
   'direKills': 22,
   'duration': 1976,
   'endDate': 1499660138,
   'firstBloodTime': 162,
   'gameMode': 22,
   'gameResult': 1,
   'gameVersionId': 0,
   'id': 3304258209,
   'isStats': False,
   'lobbyType': 7,
   'parsedDate': 1499660887,
   'players': [{'avatar': 'https://steamcdn-a.akamaihd.net/steamcommunity/public/images/avatars/65/657e96497fffece854f1a5b8cf0d9a766e81ea8e.jpg',
     'avatarFull': 'https://steamcdn-a.akamaihd.net/steamcommunity/public/images/avatars/65/657e96497fffece854f1a5b8cf0d9a766e81ea8e_full.jpg',
     'avatarMedium': 'https://steamcdn-a.akamaihd.net/steamcommunity/public/images/avatars/65/657e96497fffece854f1a5b8cf0d9a766e81ea8e_medium.jpg',
     'award': 0,
     'expPerMinute': 283,
     'goldPerMinute': 352,
     'goldSpent': 8990,
     'hero': 92,
     'heroDamage': 5458,
     'heroHealing': 0,
     'imp': 53,
     'isRadiant': True,
     '

In [7]:
host = "https://api.stratz.com/api/v1/match/3304258209"
rs = (grequests.get(host),)
x = grequests.map(rs)
data = x[0].json()

In [8]:
keys = data.keys()

In [9]:
keys

dict_keys(['regionId', 'parsedDate', 'runeEvents', 'didRadiantWin', 'firstBloodTime', 'towerDeathEvents', 'lobbyType', 'replaySalt', 'id', 'gameMode', 'radiantKills', 'roshanEvents', 'challenge', 'endDate', 'wardEvents', 'isStats', 'avgImp', 'direKills', 'duration', 'buildingEvents', 'tier', 'rank', 'gameVersionId', 'players', 'courierEvents', 'bracket', 'gameResult', 'clusterId'])

In [10]:
data['players'][0].keys()

dict_keys(['abilityUsedEvents', 'hero', 'numDeaths', 'heroTimeDetail', 'purchaseEvents', 'imp', 'item2', 'avatar', 'buyBackEvents', 'expPerMinute', 'playerUpdateBattleEvents', 'heroDamage', 'deathEvents', 'healEvents', 'numLastHits', 'towerDamage', 'numKills', 'name', 'numDenies', 'inventoryEvent', 'playerUpdateLevelEvents', 'level', 'award', 'slot', 'role', 'soloRank', 'playerUpdateGoldEvents', 'partyRank', 'killEvents', 'playerUpdatePositionEvents', 'csEvents', 'goldEvents', 'steamId', 'profileUrl', 'playerUpdateHealthEvents', 'goldPerMinute', 'avatarFull', 'goldSpent', 'playerUpdateAttributeEvents', 'item5', 'abilityActiveLists', 'lane', 'item1', 'towerDamageEvents', 'item3', 'item0', 'item4', 'heroHealing', 'numAssists', 'isRadiant', 'avatarMedium', 'heroDamageEvents', 'abilityLearnEvents', 'experienceEvents', 'leaverStatus', 'assistEvents'])

In [11]:
print(data['players'][0]['role'])
print(data['players'][0]['lane'])
print(data['players'][0]['hero'])
print(data['players'][0]['purchaseEvents'])

1
1
92
[{'time': -89, 'item': 188}, {'time': -89, 'item': 43}, {'time': -89, 'item': 29}, {'time': -89, 'item': 44}, {'time': -89, 'item': 38}, {'time': -85, 'item': 46}, {'time': -84, 'item': 46}, {'time': 187, 'item': 84}, {'time': 193, 'item': 42}, {'time': 236, 'item': 42}, {'time': 384, 'item': 42}, {'time': 389, 'item': 244}, {'time': 389, 'item': 27}, {'time': 432, 'item': 27}, {'time': 440, 'item': 214}, {'time': 509, 'item': 46}, {'time': 528, 'item': 42}, {'time': 603, 'item': 46}, {'time': 719, 'item': 46}, {'time': 720, 'item': 42}, {'time': 809, 'item': 25}, {'time': 811, 'item': 182}, {'time': 816, 'item': 42}, {'time': 882, 'item': 46}, {'time': 1059, 'item': 46}, {'time': 1264, 'item': 64}, {'time': 1264, 'item': 65}, {'time': 1277, 'item': 46}, {'time': 1533, 'item': 46}, {'time': 1758, 'item': 23}, {'time': 1758, 'item': 21}, {'time': 1758, 'item': 22}, {'time': 1760, 'item': 60}, {'time': 1760, 'item': 108}, {'time': 1919, 'item': 42}]


**Need to get steamIDs for players by going through the matchIDs of high level games**

In [12]:
def nextBatchOfMatches(idList):
    queryParams = {"matchId":idList, 
          "include": ["Player"],
         "gameMode": [2, 22],
         "lobbyType": [2, 7]}
    x = requests.get(host, params = queryParams)
    x = x.json()
    for entry in x['results']:
        for player in entry['players']:
            playerSet.add(player['steamId'])

In [13]:
if os.path.exists('playerset.txt'):
    playerSet = np.loadtxt('playerset.txt', dtype=np.int64)
else:
    playerSet = set()
    allMatches = np.loadtxt('matchids.txt', dtype=np.int64)
    allMatches = split(allMatches, 10)
    for matches in allMatches:
        nextBatchOfMatches(matches)
    playerSet = list(playerSet)
    np.savetxt('playerset.txt', playerSet, delimiter=',', fmt = "%.0f")

With player set, we can obtain matches. For the players above, get the matches these players play and also categorize the players and heroes

In [14]:
# 8 is jug and 74 is invoker and 106 is ember
def playerMatchObtain(player, heroList = [8, 74, 106]):
    host = "https://api.stratz.com/api/v1/match"
    queryParams = {"steamId": player, 
                   "heroId": heroList,
                   "gameMode": [2, 22],
                   "lobbyType": [2, 7],
                   "gameVersion": 79, 
                  "take": 100}
    x = requests.get(host, params = queryParams)
    x = x.json()
    for entry in x['results']:
        matchSet.add(entry['id'])

In [15]:
matchSet = set()
if os.path.exists('matchset.txt'):
    matchSet = np.loadtxt('matchset.txt', dtype=np.int64)
else:
    allplayers = np.loadtxt('playerset.txt', dtype=np.int64)
    for player in allplayers:
        playerMatchObtain(player, heroList=[74])
    matchSet = list(matchSet)
    np.savetxt('matchset.txt', matchSet, delimiter=',', fmt = "%.0f")

In [16]:
len(playerSet)

589

In [17]:
len(matchSet)

1542

** Time to Parse these Matches**

In [30]:
def analyzeMatchesForFeatures(matchList, heroesToTestItems = [8, 74, 106]):
    hostInitial = "https://api.stratz.com/api/v1/match/"
    rs = [grequests.get(hostInitial + str(match)) for match in matchList]
    x = grequests.map(rs)
    return x

In [31]:
x = analyzeMatchesForFeatures(matchSet[:10])

[<grequests.AsyncRequest object at 0x7f4ba9e9eb38>, <grequests.AsyncRequest object at 0x7f4ba9eeb320>, <grequests.AsyncRequest object at 0x7f4ba9eeba90>, <grequests.AsyncRequest object at 0x7f4ba9eeb278>, <grequests.AsyncRequest object at 0x7f4ba9ea0eb8>, <grequests.AsyncRequest object at 0x7f4ba9ea4390>, <grequests.AsyncRequest object at 0x7f4ba9eaed30>, <grequests.AsyncRequest object at 0x7f4ba9e7d0f0>, <grequests.AsyncRequest object at 0x7f4ba9e7d438>, <grequests.AsyncRequest object at 0x7f4ba9e7d780>]
